In [1]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

%matplotlib inline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# checing if o3d is instaleld
o3d.__version__

'0.17.0'

#### Visualise raw point cloud

In [ ]:
def segment_person(pcd, eps=0.02, min_points=10):
    """
    Segments the person from the point cloud using DBSCAN.

    Args:
        pcd (open3d.geometry.PointCloud): The input point cloud.
        eps (float): The maximum distance between two samples for one to be considered as in the neighborhood of the other.
        min_points (int): The number of samples in a neighborhood for a point to be considered as a core point.

    Returns:
        open3d.geometry.PointCloud: The point cloud of the segmented person.
    """
    points = np.asarray(pcd.points)

    # Run DBSCAN on the point cloud
    labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points))

    # Find the label of the largest cluster (assumed to be the person)
    person_label = np.argmax(np.bincount(labels[labels != -1]))

    # Create a point cloud for the person
    person_points = np.array(pcd.points)[labels == person_label]
    person_colors = np.array(pcd.colors)[labels == person_label]
    person_pcd = o3d.geometry.PointCloud()
    person_pcd.points = o3d.utility.Vector3dVector(person_points)
    person_pcd.colors = o3d.utility.Vector3dVector(person_colors)
    
    return person_pcd

def minimalBoundingBox(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
    Applies a minimal bounding box, isolating the human subject from the scene.
    This helps to remove majority of the unnecessary points.
    
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
    
    Returns:
        type: o3d.cpu.pybind.geometry.PointCloud'     
    '''
    
    points = np.asarray(pcd.points)
    
    min_x, min_y, min_z = points.min(axis=0)[:3]
    max_x, max_y, max_z = points.max(axis=0)[:3]

    # std dev in each dimension
    std_dev_x, std_dev_y, std_dev_z = np.std(points, axis=0)[:3]

    # arbitrary value -- adjust if needed
    factor = 0.1 

    padding_x = factor * std_dev_x
    padding_y = factor * std_dev_y

    # set to 0 cuz only want to adjust x/y dimensions
    padding_z = 0 

    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(min_x - padding_x, min_y - padding_y, min_z - padding_z), 
                                               max_bound=(max_x + padding_x, max_y + padding_y, max_z + padding_z))
    bbox.get_minimal_oriented_bounding_box()
    bbox.color = [1,0,0]

    # Select points within the bounding box
    pcd_in_bbox = pcd.crop(bbox)
    
    o3d.visualization.draw_geometries([bbox, pcd_in_bbox])
    
    return pcd_in_bbox

def isolate_person(pcd, eps=0.02, min_points=10):
    """
    Segments the person from the point cloud and applies a minimal bounding box.
    """
    # Segment the person
    person_pcd = segment_person(pcd, eps, min_points)

    # Apply the bounding box
    pcd_in_bbox = minimalBoundingBox(person_pcd)

    return pcd_in_bbox

o3d.visualization.draw_geometries([isolate_person(pcd)])

In [19]:
pcd = o3d.io.read_point_cloud('./data/P002 2022-01-25 01_24_48.ply')

In [21]:


def bounding_box():
    point_cloud = np.asarray(pcd.points)
    
    # Define the bounding box
    min_x = np.min(point_cloud[:, 0])
    max_x = np.max(point_cloud[:, 0])
    min_y = np.min(point_cloud[:, 1])
    max_y = np.max(point_cloud[:, 1])
    min_z = np.min(point_cloud[:, 2])
    max_z = np.max(point_cloud[:, 2])

    # Expand the bounding box slightly
    padding = 0.5  # for example, 10% of the range

    min_x = padding * min_x
    max_x = padding * max_x
    min_y = padding * min_y
    max_y = padding * max_y
    min_z = padding * min_z

    # Filter the points
    human_points = point_cloud[
        (point_cloud[:, 0] >= min_x) & (point_cloud[:, 0] <= max_x) &
        (point_cloud[:, 1] >= min_y) & (point_cloud[:, 1] <= max_y) &
        (point_cloud[:, 2] >= min_z) & (point_cloud[:, 2] <= max_z)
    ]
    
    human_colors = np.array(pcd.colors)[
        (point_cloud[:, 0] >= min_x) & (point_cloud[:, 0] <= max_x) &
        (point_cloud[:, 1] >= min_y) & (point_cloud[:, 1] <= max_y) &
        (point_cloud[:, 2] >= min_z) & (point_cloud[:, 2] <= max_z)
    ]


    ###
    visualized_point_cloud = o3d.geometry.PointCloud()
    visualized_point_cloud.points = o3d.utility.Vector3dVector(human_points)
    visualized_point_cloud.colors = o3d.utility.Vector3dVector(human_colors)
    
    obb = visualized_point_cloud.get_minimal_oriented_bounding_box()
    obb.color = [0,0,1]
    # abb = visualized_point_cloud.get_oriented_bounding_box()
    # abb.color = [0,1,0]
    o3d.visualization.draw_geometries([visualized_point_cloud, obb])
    

    print("done")

bounding_box()

done


### Vertex normal estimation

- `estimate_normals` function finds adjacent points and calculates the principal axis of the adjacent points using covariance analysis
    - `radius` = search radius, `max_nn` = maximum nearest neighbour
    - so radius = 0.1 (10cm search radius) and max_nn = 30 (only considers 30 neighbours)
- estimates normal vector (vector perpindicular to tangent plane) for each point in point cloud.
- sensitive to noise and variations & computationally expensive

In [30]:
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

downpcd.normals[0]